In [1]:
# List of modules that are not installed in the course
!pip install OpenPermID
!pip install geocoder
!pip install matplotlib
!pip install xlrd==1.2.0
!pip install xlsxwriter
!pip install sklearn
!pip install ipywidgets

You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\wpare\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
parentDirectory = 'https://s3.console.aws.amazon.com/s3/buckets/s3groupsweden/'

# Downloading all bonds ever owned in CSPP

All bonds are downloaded from the CSPP website. They are saved to an excel file for easy retrieval and saved to a dataframe for analysis in this notebook.

In [4]:
from functions import downloadECBBonds

In [5]:
holdingsECB = downloadECBBonds.download_ECB_Bonds(parentDirectory)

HTTPError: HTTP Error 403: Forbidden

# Read all the Eikon data

Additional data on the bonds from the EIKON database was downloaded and saved to .txt files, which are imported here. There was also a list of eligible bonds in this database. The same data for these bonds was downloaded and is imported as well.

In [6]:
from functions import readEikonData
eikon_data_complete = readEikonData.get_eikon_data_complete()
eligible_complete = readEikonData.get_data_eligible_complete() # read all data from eligible universe

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\wpare\\OneDrive\\Documenten\\GitHub\\ECB-Bond-Purchases\\functions\\readEikonData.py\\..\\../Data/EIKON/holdingsECBEnvironment.txt'

Some bonds bought by the ECB are also mentioned in the eligible universe. The percentage of bonds appearing in both sets is calculated here. 

In [ ]:
overlap = eikon_data_complete[(eikon_data_complete["ISIN"].isin(eligible_complete["ISIN"]))]
print("the percentage of bonds bought by ECB in eligible universe that we have info on:    ", 
      overlap.shape[0]/eligible_complete.shape[0])

The date that a bond is bought by the ECB is also of interest. This is saved to a dataframe with the following function.

In [ ]:
years_issuer_bought = readEikonData.get_dates_data_frame()

# Green Bond Analysis based on EIKON green bond flags

The EIKON database is compared with the Euronext database to check which has the most information on green bonds.

In [ ]:
from functions import greenbonds
greenbonds.compareGreenbondsEuronextvsEIKON()

It appears that EIKON has the most green bond flags, so this data is used in the rest of this section. The number of green bonds bought in a year can now be compared with the total number of bonds bought in that same year.

In [ ]:
greenbonds.get_percentages_greenbonds_on_bonds_per_year()

A graphical representation of this table is made here.

In [ ]:
greenbonds.draw_spaghetti_plot_greenbonds()

# Company Analysis based on ESG scores

## Cleaning ESG Data
The ESG data retrieved from the EIKON database is cleaned first. The functions returns the number of bonds that have no ESG data available, and how many had missing data that is filled in using linear interpolation.

In [ ]:
from functions import ESG
print("for eligible universe:")
esg_company_data_eligible = ESG.cleaning_esg_data(eligible_complete)
print("for holdings in the CSPP:")
esg_company_data_holdings = ESG.cleaning_esg_data(eikon_data_complete)

## ESG Evolution of holdings through the years
The data on the ESG scores of the ECB's holdings is split up per year (based on when they first appeared in the weekly overview) and compared with the evolution of the ESG scores of the companies selling bonds eligible for purchase by the ECB. 

In [ ]:
ESG.average_company_esg_score_plot(years_issuer_bought, esg_company_data_holdings,esg_company_data_eligible)

# Sector Analysis based on PermID sector data

## Get all sector data from PermID
The PermID API is used to retrieve information on the sector and industry to which the bonds belong.

In [ ]:
from functions import permidSectors
sector_mappings = permidSectors.get_sector_mappings(holdingsECB)
sector_mappings.dropna(inplace = True)

## Represent industries and sectors

Create a pie chart to represent the number of times an industry, sector or supersector is present in the data (multiple bonds for the same company are counted separately)

In [ ]:
from functions import pieCharts

In [ ]:
pieCharts.make_pie_chart('hasPrimaryBusinessSector', sector_mappings)

In [ ]:
pieCharts.make_pie_chart('hasPrimaryEconomicSector', sector_mappings)

In [ ]:
pieCharts.make_pie_chart('hasPrimaryIndustryGroup', sector_mappings)

## Which industries and sectors are green?

We made a dictionary containing all the sectors and a value that represents whether we think they are green, neutral, or grey. This dictionary with greenness of all PermID sectors is mapped with the ones in which ECB invested.

In [ ]:
from functions import sectors
sector_green_dict = sectors.get_sector_green_dict()

primary_business_sector = pieCharts.get_all_sectors("hasPrimaryBusinessSector", sector_mappings)
primary_business_sector = sectors.map_green_dict_to_data_frame(primary_business_sector)
sector_mappings["Greenness"] = sector_mappings["hasPrimaryBusinessSector"].dropna().apply(lambda x: sector_green_dict[x])

sectors.set_global_sector_variables(primary_business_sector, sector_mappings, years_issuer_bought)

A spaghetti plot is now created to compare the evolution of bonds bought by the ECB, grouped by there greenness according to our dictionary.

In [ ]:
# Drawing a sphagetti plot by grouping sectors according to greenness
levels_of_greenness = pieCharts.get_all_sectors("hasPrimaryBusinessSector", sectors.get_sector_mappings_modified())
sectors.draw_spaghetti_plot_with_grouped_sectors(levels_of_greenness)

Here, an interactive graph is created using a widget. The sectors of interest can be selected, to allow an easy comparison.

In [ ]:
sectors.display_widget()